In [ ]:
import numpy as np
import heapq

class Puzzle:
    def __init__(self, initial_state):
        self.initial_state = initial_state
        self.goal_state = np.array([[1, 2, 3],
                                    [4, 5, 6],
                                    [7, 8, 0]])

    def is_goal(self, state):
        return np.array_equal(state, self.goal_state)

    def __str__(self):
      return str(self.initial_state)

class PuzzleNode:
    def __init__(self, state, cost, path):
        self.state = state
        self.cost = cost
        self.path = path

    def __lt__(self, other):
        return self.cost < other.cost

class SearchBase:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.explored = set()

    def find_blank(self, state):
        return np.argwhere(state == 0)[0]

    def apply_action(self, state, action):
        blank = self.find_blank(state)
        new_blank = blank + action

        if (new_blank >= 0).all() and (new_blank < 3).all():
            new_state = state.copy()
            new_state[blank[0], blank[1]], new_state[new_blank[0], new_blank[1]] = \
                new_state[new_blank[0], new_blank[1]], new_state[blank[0], blank[1]]
            return new_state
        else:
            return None

class GreedyBestFirstSearch(SearchBase):
    def heuristic(self, state):
        distance = 0
        for r in range(3):
            for c in range(3):
                if state[r][c] != 0:
                    goal_pos = np.argwhere(self.puzzle.goal_state == state[r][c])[0]
                    distance += abs(r - goal_pos[0]) + abs(c - goal_pos[1])
        return distance

    def solve(self):
        frontier = []
        heapq.heappush(frontier, PuzzleNode(self.puzzle.initial_state, self.heuristic(self.puzzle.initial_state), [self.puzzle.initial_state]))

        while frontier:
            current_node = heapq.heappop(frontier)
            current_state = current_node.state
            current_path = current_node.path

            if self.puzzle.is_goal(current_state):
                return current_path

            self.explored.add(tuple(current_state.flatten()))

            for action in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
                new_state = self.apply_action(current_state, action)
                if new_state is not None and tuple(new_state.flatten()) not in self.explored:
                    new_path = current_path + [new_state]
                    heapq.heappush(frontier, PuzzleNode(new_state, self.heuristic(new_state), new_path))

        return None


initial_state = np.array([[1, 2, 3],
                          [4, 5, 6],
                          [7, 0, 8]])

puzzle = Puzzle(initial_state)
print("Initial State:")
print(puzzle,"\n")

print("Greedy Best-First Search Result:")
greedy_solver = GreedyBestFirstSearch(puzzle)
result_greedy = greedy_solver.solve()

if result_greedy:
    for i, state in enumerate(result_greedy):
        print(f"Step {i}:\n{state}\n")
else:
    print("No solution found.")


Initial State:
[[1 2 3]
 [4 5 6]
 [7 0 8]] 

Greedy Best-First Search Result:
Step 0:
[[1 2 3]
 [4 5 6]
 [7 0 8]]

Step 1:
[[1 2 3]
 [4 5 6]
 [7 8 0]]


Uniform Cost Search Result:
Step 0:
[[1 2 3]
 [4 5 6]
 [7 0 8]]

Step 1:
[[1 2 3]
 [4 5 6]
 [7 8 0]]

